In [ ]:
import cv2
from matplotlib import pyplot as plt
import tensorflow as tf

In [ ]:
vc = cv2.VideoCapture(0)
vc.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
vc.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)


frame = None
if vc.isOpened(): # try to get the first frame
    for i in range(10):
      rval, frame = vc.read()
else:
    rval = False
    
frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

plt.imshow(frame)
plt.show()

vc.release()


In [ ]:
%matplotlib widget
import numpy as np
import ipywidgets as widgets
import transform
import tensorflow_addons as tfa
import common

fig, ax = plt.subplots(ncols=2, figsize=(6, 4))
source_axis = ax[0]
dest_axis = ax[1]

tl_x = widgets.IntSlider(min=0, max=1279, step=1, value=300)
tl_y = widgets.IntSlider(min=0, max=719, step=1, value=100)
tr_x = widgets.IntSlider(min=0, max=1279, step=1, value=1279-300)
tr_y = widgets.IntSlider(min=0, max=719, step=1, value=100)
bl_x = widgets.IntSlider(min=0, max=1279, step=1, value=300)
bl_y = widgets.IntSlider(min=0, max=719, step=1, value=719-100)
br_x = widgets.IntSlider(min=0, max=1279, step=1, value=1279-300)
br_y = widgets.IntSlider(min=0, max=719, step=1, value=719-100)

loaded_config = common.load_config()
if hasattr(loaded_config, 'camera_keypoints'):
  [[tl_x.value, tl_y.value], [tr_x.value, tr_y.value], [bl_x.value, bl_y.value], [br_x.value, br_y.value]] = loaded_config.camera_keypoints


@widgets.interact(tl_x=tl_x, tl_y=tl_y, tr_x=tr_x, tr_y=tr_y, bl_x=bl_x, bl_y=bl_y, br_x=br_x, br_y=br_y)
def display_transformed(tl_x, tl_y, tr_x, tr_y, bl_x, bl_y, br_x, br_y):
  corners = [(tl_x, tl_y), (tr_x, tr_y), (bl_x, bl_y), (br_x, br_y)]
  t = transform.keypoints_to_transform(640, 448, *corners)
  transformed_image = tfa.image.transform(frame, t, interpolation='bilinear', fill_value=255, output_shape=(448, 640))
  source_image = frame.copy()
  cv2.line(source_image, (tl_x, tl_y), (tr_x, tr_y), (0, 255, 0), thickness=3)
  cv2.line(source_image, (br_x, br_y), (tr_x, tr_y), (0, 255, 0), thickness=3)
  cv2.line(source_image, (bl_x, bl_y), (br_x, br_y), (0, 255, 0), thickness=3)
  cv2.line(source_image, (bl_x, bl_y), (tl_x, tl_y), (0, 255, 0), thickness=3)

  source_axis.clear()
  dest_axis.clear()
  source_axis.imshow(source_image)
  dest_axis.imshow(transformed_image)
  print(corners)

save_button = widgets.Button(description='Save')
display(save_button)

def click_handler(keypoints):
    def handler(btn):
        config = common.load_config()
        print(config)
        config.camera_keypoints = [(x.value, y.value) for x,y in keypoints]
        print(config)
        common.save_config(common.to_dictionary(config))
    return handler

save_button.on_click(click_handler([(tl_x, tl_y), (tr_x, tr_y), (bl_x, bl_y), (br_x, br_y)]))